In [69]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import ase
#import gudhi
import os
from glob import glob

from numba import jit
from tqdm import tqdm_notebook

from sklearn.externals import joblib as jl

import warnings
warnings.filterwarnings("ignore")

In [2]:
class HorizontalDisplay:
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        template = '<div style="float: left; padding: 10px;">{0}</div>'
        return "\n".join(template.format(arg._repr_html_())
                         for arg in self.args)

In [44]:
@jit(nopython=True)
def normalize(v):
    return v / np.linalg.norm(v)

In [42]:
normalize(np.array([1.,2.,3.]))

TypingError: Failed at nopython (nopython frontend)
cannot unify array(float32, 1d, C) and float64 for 'tmp', defined at <ipython-input-41-be1337693c38> (4)
File "<ipython-input-41-be1337693c38>", line 6
[1] During: typing of assignment at <ipython-input-41-be1337693c38> (6)

In [62]:
train = pd.read_csv("../input/train.csv", engine='c')
test = pd.read_csv("../input/test.csv", engine='c')
structs = pd.read_csv("../input/structures.csv", engine='c')

In [49]:
structs.describe()

,atom_index,x,y,z
count,2.358657e+06,2.358657e+06,2.358657e+06,2.358657e+06
mean,8.757349e+00,9.489178e-02,-3.337381e-01,6.241504e-02
std,5.592487e+00,1.655271e+00,1.989152e+00,1.445870e+00
min,0.000000e+00,-9.234889e+00,-9.933938e+00,-9.134765e+00
25%,4.000000e+00,-8.746097e-01,-1.826156e+00,-8.424896e-01
50%,9.000000e+00,5.183615e-02,-4.035932e-01,1.093207e-02
75%,1.300000e+01,1.116101e+00,1.373660e+00,9.394357e-01
max,2.800000e+01,9.382240e+00,1.018196e+01,7.894733e+00


In [5]:
name = "dsgdb9nsd_000001" # from train

HorizontalDisplay(
    train[train.molecule_name == name],
    structs[structs.molecule_name == name]
)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095


In [46]:
@jit(nopython=True)
def two_point_standardization(struct, idx0, idx1):
    
    coo0 = struct[idx0]
    coo1 = struct[idx1]
    
    if np.linalg.norm(coo1) < np.linalg.norm(coo0):
        
        tmp = coo0
        coo0  = coo1
        coo1 = tmp
    
    normal = (coo1 - coo0) / 2
    
    normal_unit = normalize(normal)
    
    vec0 = coo0 + normal
    
    # Plane Equation
    d = -vec0 @ normal
    plane = lambda x, y: (-normal[0] * x - normal[1] * y - d) / normal[2] if 0 != normal[2] else 0.
    #
    
    # Make orthonormal basis on plane
    u_x = 1
    u_y = 0
    
    u = np.array([u_x, u_y, plane(u_x + vec0[0], u_y + vec0[1]) - vec0[2]])
    
    ## Cross product
    a1, a2, a3 = u
    b1, b2, b3 = normal
    
    v = np.array(
        [
            a2 * b3 - a3 * b2,
            a3 * b1 - a1 * b3,
            a1 * b2 - a2 * b1
        ]
    ) 
    ##
    
    u_unit = normalize(u)
    v_unit = normalize(v)
    #

    # Change of basis
    basis_matrix = np.vstack((normal_unit, u_unit, v_unit)).T    
    transform_matrix = np.linalg.inv(basis_matrix)
    
    return transform_matrix @ (struct - vec0).T
    #

In [75]:
@jit()
def get_classified_idx(struct, tgt0, tgt1): 

    standardized = np.around(two_point_standardization(struct.values, tgt0, tgt1), decimals=14)
    
    print(standardized)
    
    struct['x_changed'] = standardized[0]
    struct['norm'] = np.linalg.norm(standardized, axis=0)

    if 0 < struct.x_changed[tgt0]:
        struct.x_changed *= -1

    I = struct[0 > struct.x_changed]
    II = struct[0 < struct.x_changed]
    
    return \
        l2s(I[struct.x_changed[tgt0] >= I.x_changed].sort_values(by=['norm'], ascending=False).index.values), \
        l2s(I[struct.x_changed[tgt0] <= I.x_changed].sort_values(by=['norm'], ascending=False).index.values), \
        l2s(II[struct.x_changed[tgt1] >= II.x_changed].sort_values(by=['norm'], ascending=False).index.values), \
        l2s(II[struct.x_changed[tgt1] <= II.x_changed].sort_values(by=['norm'], ascending=False).index.values)

In [74]:
l2s = lambda l: str(list(l))[1:-1]
s2l = lambda s: list(map(int, s.split(', ')))

seg_id = 0
singular_avoiding = 10

inf_to_tgt_I = []
center_to_tgt_I = []
center_to_tgt_II = []
inf_to_tgt_II = []

for idx in tqdm_notebook(np.array_split(train.index, 4)[seg_id]):

    name = train.molecule_name[idx]
    tgt0 = int(train.atom_index_0[idx])
    tgt1 = int(train.atom_index_1[idx])
    target_frame = structs[structs.molecule_name == name].reset_index()
    
    classified = get_classified_idx(target_frame[['x','y','z']]+singular_avoiding, tgt0, tgt1)
    
    inf_to_tgt_I.append(classified[0])
    center_to_tgt_I.append(classified[1])
    center_to_tgt_II.append(classified[2])
    inf_to_tgt_II.append(classified[3])
    
jl.dump(inf_to_tgt_I, f'inf_to_tgt_I_{seg_id}.pkl')
jl.dump(center_to_tgt_I, f'center_to_tgt_I_{seg_id}.pkl')
jl.dump(center_to_tgt_II, f'center_to_tgt_II{seg_id}.pkl')
jl.dump(inf_to_tgt_II, f'inf_to_tgt_II{seg_id}.pkl')

A Jupyter Widget

KeyboardInterrupt: 

In [82]:
idx = 128382
    
name = train.molecule_name[idx]
tgt0 = int(train.atom_index_0[idx])
tgt1 = int(train.atom_index_1[idx])
target_frame = structs[structs.molecule_name == name].reset_index()
classified = get_classified_idx(target_frame[['x','y','z']]+10, tgt0, tgt1)

print(classified)
classified = get_classified_idx(target_frame[['x','y','z']], tgt0, tgt1)

print(classified)

[[-1.36169339 -0.4174461   0.86083976  1.77048544  2.98400448  3.3542545
   2.32254456  1.29403376 -0.98664231 -2.3189436  -1.54464033 -0.81307842
   1.36169339  0.74705284  4.32916601  2.14512247]
 [-0.          0.06732314 -0.56391974 -0.56222389 -0.12168254 -0.39397375
  -0.99096389 -1.11241278  0.59870924  0.42686496 -1.022304   -0.38392787
  -0.         -1.60351894 -0.13538624 -1.36472091]
 [ 0.          1.10648347  0.79249869  1.97354853  2.06837794  3.38096917
   4.01770779  3.12367567 -0.83790059  0.31152714 -0.37893272  1.9243207
   0.          0.42484429  3.76211316  5.01108715]]
('14, 15, 5, 6, 4, 3, 12', '7, 13, 12, 2', '11, 8, 0, 1', '9, 10, 0')
[[-1.36169339 -0.4174461   0.86083976  1.77048544  2.98400448  3.3542545
   2.32254456  1.29403376 -0.98664231 -2.3189436  -1.54464033 -0.81307842
   1.36169339  0.74705284  4.32916601  2.14512247]
 [ 0.          0.06732314 -0.56391974 -0.56222389 -0.12168254 -0.39397375
  -0.99096389 -1.11241278  0.59870924  0.42686496 -1.022304   

In [83]:
np.around(1.123e-13, decimals=13)

1e-13